# Hebb learning
李欣哲 2022141460113

## 1. Hebb学习算法的Python实现
### 前置小工具

In [1]:
import random
import numpy as np

# 门 (1保留，0置0)
def mask_image(image, mask_image):
    # return image * mask_image
    return np.where(mask_image, image, 0)

# 反转门 (1保留，0翻转)
def toggle_mask(image, mask_image):
    return np.where(mask_image == np.ones_like(mask_image), image, -image)

# 部分覆盖 (1保留img1，0保留img2)
def cover_mask(image1, image2, mask_image):
    return np.where(mask_image == np.ones_like(mask_image), image1, image2)

# "同或" (==保留，!=0)
def XNOR(image1, mask_image):
    return np.where(image1 == mask_image, image1, 0)

# 随机生成mask (mask_prob概率为1，否则为0)
def get_randomly_mask(image, mask_prob):
    return np.where(np.random.rand(*image.shape) < mask_prob, 1, 0)

# 添加正态分布噪声
def add_gaussian_noise(image, mean, std_dev):
    return image + np.random.normal(mean, std_dev, image.shape)


# 预备masks
mask_data = np.array([
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]], # 0:上半0
    [[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]], # 1:下半0
    [[1,1,1,1,1],[0,1,1,1,1],[0,0,1,1,1],[0,0,0,1,1],[0,0,0,0,1],[0,0,0,0,0]], # 2:下三角0
    [[0,0,0,0,0],[1,0,0,0,0],[1,1,0,0,0],[1,1,1,0,0],[1,1,1,1,0],[1,1,1,1,1]], # 3:上三角0
    [[1,1,1,1,1],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]], # 4:
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[1,1,1,1,1]], # 5:
    [[1,1,1,1,1],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[1,1,1,1,1]], # 6:
])

### 准备数据

In [2]:
import numpy as np

# 将训练数据转换为向量
train_data = np.array([
    [[-1,1,1,1,-1], [1,-1,-1,-1,1], [1,-1,-1,-1,1], [1,-1,-1,-1,1], [1,-1,-1,-1,1], [-1,1,1,1,-1]],  # 0
    [[-1,1,1,-1,-1],[-1,-1,1,-1,-1],[-1,-1,1,-1,-1],[-1,-1,1,-1,-1],[-1,-1,1,-1,-1],[-1,-1,1,-1,-1]],  # 1
    [[1,1,1,-1,-1], [-1,-1,-1,1,-1],[-1,-1,-1,1,-1],[-1,1,1,-1,-1], [-1,1,-1,-1,-1],[-1,1,1,1,1]]   # 2
])

# 测试数据 (残缺的图片0)
# 要求中只给了一张

test_data = np.array([
    0.1 * train_data[0],
    mask_image(train_data[0], mask_data[0]),
    mask_image(train_data[0], mask_data[1]),
    mask_image(train_data[0], mask_data[2]),
    XNOR(train_data[0], train_data[2]), # 0与2图片取"同或"
    cover_mask(train_data[0], XNOR(train_data[0], train_data[2]), mask_data[4]), # 0与2图片取"同或"，保留0的上行
    cover_mask(train_data[0], XNOR(train_data[0], train_data[2]), mask_data[5]), # 0与2图片取"同或"，保留0的下行 (基本可以还原)
    cover_mask(train_data[0], XNOR(train_data[0], train_data[2]), mask_data[6]), # 0与2图片取"同或"，保留0的上下两行 (同上)
    toggle_mask(train_data[0], np.zeros_like(train_data[0])), # 翻转
    # 含随机噪声图片
    # 随机翻转
    toggle_mask(train_data[0], get_randomly_mask(train_data[0], 0.7)), # 0.3
    toggle_mask(train_data[0], get_randomly_mask(train_data[0], 0.6)), # 0.4
    # 正态分布
    add_gaussian_noise(train_data[0], 0, 1),
    add_gaussian_noise(train_data[0], 0, 2),
    add_gaussian_noise(train_data[0], 0, 3),
    add_gaussian_noise(train_data[0], random.uniform(-0.7,0.7), random.randrange(2, 5)),
    # 其他数据
    [[-1,1,1,1,-1], [1,-1,-1,-1,1], [-1,1,1,1,-1], [1,-1,-1,-1,1], [1,-1,-1,-1,1], [-1,1,1,1,-1]], # 8
    [[-1,1,1,1,-1], [-1,-1,-1,-1,1], [-1,1,1,1,1], [-1,-1,-1,-1,1], [-1,-1,-1,-1,1], [-1,1,1,1,-1]], # 3
    [[-1,1,1,1,1],[-1,-1,-1,-1,1],[-1,-1,-1,1,-1],[-1,-1,1,-1,-1],[-1,1,-1,-1,-1],[1,-1,-1,-1,-1]], # 7
    [[1,1,1,1,1],[-1,-1,-1,-1,1],[-1,-1,-1,1,-1],[-1,-1,1,-1,-1],[-1,1,-1,-1,-1],[1,-1,-1,-1,-1]] # 7（变种）
])



### 定义网络

In [3]:
class Hebb_learning:
    # 建议将__init__折叠
    def __init__(self):
        self.weight = None
        self.weight_diagonal_0 = None
        self.alpha = 1
        self.num_features = None
        self.enable_self_association = False
        
        self.SIGNS = [' ','▪','▫','\033[34m■\033[0m','\033[32m□\033[0m', ' ', '■','□'] # 常量
        
        # 输出用的函数，不用看
        def glance(self, data, data_2=None, data_3=None, pre_str=None):
            # 
            def print_img(img):
                def get_sign(x):
                    return self.SIGNS[max(int(x>0)*1,int(x<0)*2,int(x>1)*3,int(x<-1)*4,int(x==0)*5,int(x==1)*6,int(x==-1)*7)]
                for row in img:
                    print(' '.join([get_sign(x) for x in row]))
                print()
            data_img = np.concatenate((data, np.zeros_like(data), (data_2 if data_2 else np.zeros_like(data)), np.zeros_like(data), (data_3 if data_3 else np.zeros_like(data))), axis = 2)

            if isinstance(pre_str, str):
                print(pre_str)
            for img in data_img:
                print_img(img)
        # 有地方可以复用，于是定义为self函数吧 # 对于with_self_association() 以及 without_self_association()
        def hebb_recall(data, weight):
            output_data = []
            for img in data:
                vec_img = img.flatten()
                vec_output = None

                output = np.dot(weight, vec_img)
                vec_output = np.sign(output) # 转换为-1 or 1 or 0
                output_data.append(vec_output.reshape(data.shape[1], data.shape[2]))
            return output_data
        
        self.__glance = glance
        self.__hebb_recall = hebb_recall
    
    def train(self, data, alpha=1, glance_data = False): # hebb_train
        self.alpha = alpha
        self.num_features = data.shape[1] * data.shape[2]
        self.weight = np.zeros((self.num_features, self.num_features))
        if glance_data:
            self.__glance(self, data)

        for img in data:
            vec_img = img.flatten()
            self.weight += np.outer(vec_img, vec_img)
            # self.weight += self.alpha * np.outer(vec_img, vec_img) # 本来该如此
        self.weight *= self.alpha                                    # 但与最后乘alpha没区别
        self.weight_diagonal_0 = self.weight.copy()
        np.fill_diagonal(self.weight_diagonal_0, 0)

        return self.weight
    
    def test(self, data, enable_self_association = False, glance_result = True): # hebb_recall
        # 检查形状 DEBUG时用的，可以删了
        if not self.num_features == (data.shape[1] * data.shape[2]):
            print("[ERROR] The image shape of training set and test set are different.")
            return

        # 闭包 self不需要再传
        # 无自联想
        def without_self_association():
            output_data_without = self.__hebb_recall(data, self.weight_diagonal_0) # hebb_recall 不含自联想
            if glance_result:
                self.__glance(self, data, output_data_without, pre_str='\033[1;32m原图\t\t不含自联想\033[0m\n')
            return output_data_without # 不含自联想
        # 有自联想
        def with_self_association():
            output_data_with = self.__hebb_recall(data, self.weight) # hebb_recall 含自联想
            output_data_without = self.__hebb_recall(data, self.weight_diagonal_0) # hebb_recall 不含自联想(对比用)
            if glance_result:
                self.__glance(self, data, output_data_without, output_data_with, pre_str='\033[1;32m原图\t\t不含自联想\t\t含自联想\033[0m\n')
            return output_data_with # 含自联想
        
        # 有无自联想
        if enable_self_association == False:
            return without_self_association()
        return with_self_association()

In [4]:
# ===== main =====
hl = Hebb_learning()

# hl.train(train_data, glance_data=True)
hl.train(train_data)

# re_test_data = hl.test(test_data)
re_test_data = hl.test(test_data, enable_self_association=True)

原图		不含自联想		含自联想

▫ ▪ ▪ ▪ ▫           □ ■ ■ ■ □           □ ■ ■ ■ □
▪ ▫ ▫ ▫ ▪           ■ □ □ □ ■           ■ □ □ □ ■
▪ ▫ ▫ ▫ ▪           ■ □ □ □ ■           ■ □ □ □ ■
▪ ▫ ▫ ▫ ▪           ■ □ □ □ ■           ■ □ □ □ ■
▪ ▫ ▫ ▫ ▪           ■ □ □ □ ■           ■ □ □ □ ■
▫ ▪ ▪ ▪ ▫           □ ■ ■ ■ □           □ ■ ■ ■ □

                    □ ■ ■ ■ □           □ ■ ■ ■ □
                    ■ □ □ □ ■           ■ □ □ □ ■
                    ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
                    ■ □ □ □ ■           ■ □ □ □ ■
                    ■ □ □ □ ■           ■ □ □ □ ■
                    □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
  □ □ □ ■           ■ □ □ □ ■ 

可以发现关闭自联想后：

- 还原**残缺或冗余**图片 的能力  下降 (e.g.第8组测试数据)

- 还原**含随机噪声**图片 的能力  上升 (e.g.第10~15组测试数据)

关闭自联想有助于将图形还原为近似于训练用的图片（容易欠拟合）

开启自联想有助于将图形还原为相同于训练用的图片 或 其组合与片段（及其容易过拟合）(e.g.第18组测试数据，输入未见过的7，还原直接与1相同了)

**注**：

- 第9~14组及之后的数据有随机性，由于没有使用种子，在我多次重新运行覆盖结果后，不能保证复现出下面一样的结果

- 其实基本看不出规律，任务简单，训练图片太少、太小

- 数据不同，展现出来的效果的随机性很大

- 而且**都过拟合**，因为weight直接做加法

## 其他
### 运行数据 记录

由于是markdown，无法保留终端中的颜色

请在编辑模式，查看下方单元格（或者c、v到**记事本**查看）

原图		不含自联想		含自联想

                    □ ■ ■ ■ □           □ ■ ■ ■ □
                    ■ □ □ □ ■           ■ □ □ □ ■
                    ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
                    ■ □ □ □ ■           ■ □ □ □ ■
                    ■ □ □ □ ■           ■ □ □ □ ■
                    □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
  □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
    □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
      □ ■           ■ □ □ □ ■           ■ □ □ □ ■
        ■           ■ □ □ □ ■           ■ □ □ □ ■
                    □ ■ ■ ■ □           □ ■ ■ ■ □

  ■ ■   □           □ ■ ■ □ □           □ ■ ■ □ □
  □ □               □ □ □ □ □           □ □ □ □ □
  □ □               □ □ □ □ □           □ □ □ □ □
      □             □ □ ■ □ □           □ □ ■ □ □
    □ □             □ □ □ □ □           □ □ □ □ □
□ ■ ■ ■             □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ □           □ ■ ■ □ □           □ ■ ■   □
  □ □                 □ □ □               □ □ □  
  □ □                 □ □ □               □ □ □  
      □               □   □               □   □  
    □ □               □ □ □               □ □ □  
□ ■ ■ ■             □ ■ ■ ■ □           □ ■ ■ ■ □

  ■ ■   □           □ ■ ■ □ □           □ ■ ■ □ □
  □ □               □ □ □ □ □           □ □ □ □ □
  □ □               □ □ □ □ □           □ □ □ □ □
      □             □ □ ■ □ □           □ □ ■ □ □
    □ □             □ □ □ □ □           □ □ □ □ □
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ □           □ ■ ■ □ □           □ ■ ■ ■ □
  □ □               ■ □ □ □ ■           ■ □ □ □ ■
  □ □               ■ □ □ □ ■           ■ □ □ □ ■
      □             ■ □ □ □ ■           ■ □ □ □ ■
    □ □             ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □

■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■

□ ■ ■ ■ ■           □ ■ ■ ■ □           □ ■ ■ ■ □
■ □ ■ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ □           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ □ ■ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ □ □           □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ ■           □ ■ ■ □ □           □ ■ ■   □
■ □ ■ □ ■           □ □ □ □ □             □ □ □  
□ □ □ □ ■           ■ □ □ □ □             □ □ □  
□ ■ ■ □ □           ■ □ □ □ ■             □   □  
■ □ □ □ □           □ □ □ □ ■             □ □ □  
□ □ □ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ ▫           □ ■ ■ ■ □           □ ■ ■ ■ □
▫ □ ▪ □ ▪           ■ □ □ □ ■           ■ □ □ □ ■
■ ▪ □ □ ▪           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ ▫ ■           ■ □ □ □ ■           ■ □ □ □ ■
▫ □ □ □ ▪           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ ▫           □ ■ ■ ■ □           □ ■ ■ ■ □

□ □ ▪ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
▫ □ □ □ ▪           ■ □ □ □ ■           ■ □ □ □ ■
■ □ ▫ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
■ ■ ■ □ ▫           ■ □ □ □ ■           ■ □ □ □ ■
■ ■ ▪ ▪ ■           ■ □ □ □ ■           ■ □ □ □ ■
■ ■ □ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □

■ □ ■ ■ ▪           ■ ■ □ ■ □           ■ ■ ■ ■ □
□ ■ ▪ □ ■           ■ □ □ ■ ■           ■ □ □ ■ ■
▫ □ □ ■ ■           ■ □ □ ■ ■           ■ □ □ ■ ■
■ ■ □ □ ■           ■ □ □ ■ ■           ■ ■ □ □ ■
■ □ □ □ □           ■ ■ □ ■ ■           ■ ■ □ □ ■
■ ■ □ □ ■           □ ■ ■ ■ ■           □ ■ ■ ■ ■

□ ■ ■ ■ □           □ ■ ■ □ □           □ ■ ■ □ □
▫ ▪ ■ □ □           ■ □ □ □ ■           □ □ □ □ □
□ ■ □ ■ □           ■ □ □ □ ■           □ □ □ □ □
▫ □ □ ▪ ▫           ■ □ ■ □ ■           □ □ ■ □ □
■ □ □ ■ ■           □ □ □ □ □           □ □ □ □ □
▪ ▫ ▫ ■ ■           □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ □           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
■ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □
□ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ □ □ □ ■           ■ □ □ □ ■           ■ □ □ □ ■
□ ■ ■ ■ □           □ ■ ■ ■ □           □ ■ ■ ■ □

□ ■ ■ ■ ■           ■ ■ ■ □ □           □ ■ ■ □ □
□ □ □ □ ■           □ □ ■ ■ □           □ □ ■ □ □
□ □ □ ■ □           □ □ ■ □ □           □ □ ■ □ □
□ □ ■ □ □           □ ■ ■ □ □           □ □ ■ □ □
□ ■ □ □ □           □ □ ■ □ □           □ □ ■ □ □
■ □ □ □ □           □ ■ ■ ■ ■           □ □ ■ □ □

■ ■ ■ ■ ■           ■ ■ ■ □ □           ■ ■ ■ □ □
□ □ □ □ ■           □ □ ■ ■ □           □ □   ■ □
□ □ □ ■ □           □ □ ■ ■ □           □ □   ■ □
□ □ ■ □ □           □ ■ ■ □ □           □ ■ ■ □ □
□ ■ □ □ □           □ ■ ■ □ □           □ ■   □ □
■ □ □ □ □           □ ■ ■ ■ ■           □   ■   ■

### 得意优化

将
```python
# 函数中的子函数
def get_sign(x):
    if x == 0:
        return ' '
    if x == 1: 
        return '■'
    if x == -1:
        return '□'
    if x > 1:
        return '\033[34m■\033[0m' # 没找到合适的符号（加粗貌似没用，只好换色）
    if x < -1:
        return '\033[34m□\033[0m' # 没找到合适的符号
    if x > 0:
        return '▪'
    if x < 0:
        return '▫'
```

优化为：

```python
# 初始化时self的变量
self.SIGNS = [' ','▪','▫','\033[34m■\033[0m','\033[34m□\033[0m', ' ', '■','□']

# 函数中的子函数
def get_sign(x): # 利用函数闭包将self.SIGNS传入该函数
    return self.SIGNS[max((x>0)*1, (x<0)*2, (x>1)*3, (x<-1)*4, (x==0)*5, (x==1)*6, (x==-1)*7)]
```

**优雅**

但是居然提醒我

```cmd
DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return self.SIGNS[max((x==1),(x==-1)*2,(x>0)*3,(x<0)*4,(x>1)*5,(x<-1)*6)]
```

只好改为

```python
def get_sign(x):
    return self.SIGNS[max(int(x>0)*1,int(x<0)*2,int(x>1)*3,int(x<-1)*4,int(x==0)*5,int(x==1)*6,int(x==-1)*7)]
# 或
def get_sign(x):
    return self.SIGNS[max(int(x>0)*1,int(x<0)*2,int(x>1)*3,int(x<-1)*4,5,int(x==1)*6,int(x==-1)*7)]
```
一堆`int()`，不优雅了